In [55]:
import pandas as pd
from itertools import zip_longest
import os

import string

from transformers import pipeline

import warnings
warnings.filterwarnings("ignore")

#! pip install transformers

In [179]:
def ner_food(result):

    """pipe = pipeline("ner", model="davanstrien/deberta-v3-base_fine_tuned_food_ner")

    result = pipe(food_name)

    result = [entity for entity in result if entity['score'] >= 0.3]

    # Iterate over the results to merge consecutive rows with the same entity
    for i in range(len(result) - 1, 0, -1):
        current_entity = result[i]["entity"]
        previous_entity = result[i - 1]["entity"].split('-')[1]

        if current_entity.split('-')[1] == previous_entity:
            if current_entity.split('-')[0] == "U":
                None
            if current_entity.split('-')[0] == "I" or current_entity.split('-')[0] == "L":
                # Append the word from the row below to the "word" column
                result[i - 1]["word"] += result[i]["word"]

                # Update the "end" value of the first row with the "end" value of the row below
                result[i - 1]["end"] = result[i]["end"]

                # Delete the current row
                del result[i]
            else:
                None"""

    # Lists to store information
    foods = []
    quantities = []
    units = []

    # Iterate over the resulting entities
    for entity in result:
        if "FOOD" in entity["entity"]:
            current_food = {"food": entity["word"], "food_start": entity["start"], "food_end": entity["end"]}
            foods.append(current_food)
        elif "QUANTITY" in entity["entity"]:
            current_quantity = {"quantity": entity["word"], "quantity_start": entity["start"], "quantity_end": entity["end"]}
            quantities.append(current_quantity)
        elif "UNIT" in entity["entity"]:
            current_unit = {"unit": entity["word"], "unit_start": entity["start"], "unit_end": entity["end"]}
            units.append(current_unit)

    # Create separate DataFrames for each type of information
    df_food = pd.DataFrame(foods)
    df_quantity = pd.DataFrame(quantities)
    df_unit = pd.DataFrame(units)

    # Check if df_quantity and df_unit are empty, and create them if needed
    if df_food.empty:
        df_food = pd.DataFrame(columns=["quantity", "quantity_start", "quantity_end"])

    if df_quantity.empty:
        df_quantity = pd.DataFrame(columns=["quantity", "quantity_start", "quantity_end"])

    if df_unit.empty:
        df_unit = pd.DataFrame(columns=["unit", "unit_start", "unit_end"])


    # Combine the DataFrames
    df_edited = pd.concat([df_food, df_quantity, df_unit], axis=1)

    # Fill NaN values with 0
    df_edited = df_edited.fillna(-1)

    for i, rows in df_edited.iterrows():
        df_edited['food'][i] = str(df_edited['food'][i]).replace("▁"," ").strip().lower().capitalize()
        df_edited['quantity'][i] = str(df_edited['quantity'][i]).replace("▁"," ").strip().lower().capitalize()
        df_edited['unit'][i] = str(df_edited['unit'][i]).replace("▁"," ").strip()


    # Replace NaN, nan, and NaN with a specific non-null value (e.g., 0) across the entire DataFrame
    df_edited.replace({pd.NaT: -1, 'Nan': -1, 'nan': -1, 'NaN': -1}, inplace=True)
    df_edited = df_edited.fillna(-1)

    df_edited['food'] = df_edited['food'].astype(str)
    df_edited['food_start'] = df_edited['food_start'].astype(int)
    df_edited['food_end'] = df_edited['food_end'].astype(int)

    #df_edited['quantity'] = df_edited['quantity'].astype(float)
    df_edited['quantity_start'] = df_edited['quantity_start'].astype(int)
    df_edited['quantity_end'] = df_edited['quantity_end'].astype(int)

    df_edited['unit'] = df_edited['unit'].astype(str)
    df_edited['unit_start'] = df_edited['unit_start'].astype(int)
    df_edited['unit_end'] = df_edited['unit_end'].astype(int)


    pd.set_option("mode.chained_assignment", None)

    #print(f"\n\n\033[1m{food_name}\033[0m\n")

    return df_edited

In [ ]:
ner_food(food_name = 'caesar salad and 100 gr steak and eggs')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.




caesar salad and 100 gr steak and eggs



,food,food_start,food_end,quantity,quantity_start,quantity_end,unit,unit_start,unit_end
0,Caesar salad,0,12,100,16,20,gr,20,23
1,Steak and eggs,23,38,-1,-1,-1,-1,-1,-1


In [75]:
pipe = pipeline("ner", model="davanstrien/deberta-v3-base_fine_tuned_food_ner")

result = pipe('potato soup,  caesar salad  3 apples')

pd.DataFrame(result)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity,score,index,word,start,end
0,B-FOOD,0.988315,1,▁potato,0,6
1,L-FOOD,0.986202,2,▁soup,6,11
2,B-FOOD,0.991229,4,▁caesar,13,20
3,L-FOOD,0.996591,5,▁salad,20,26
4,U-QUANTITY,0.979940,6,▁3,27,29
5,U-FOOD,0.992040,7,▁apples,29,36


In [ ]:
ner_food(food_name = 'potato soup, caesar salad and 3 apples')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.




potato soup, caesar salad and 3 apples



,food,food_start,food_end,quantity,quantity_start,quantity_end,unit,unit_start,unit_end
0,Potato soup,0,11,3,29,31,0,0,0
1,Caesar salad,12,25,0,0,0,0,0,0
2,Apples,31,38,0,0,0,0,0,0


In [ ]:
ner_food(food_name = 'potato soup')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.




potato soup



,food,food_start,food_end,quantity,quantity_start,quantity_end,unit,unit_start,unit_end
0,Potato soup,0,11,0,0,0,0,0,0


In [ ]:
ner_food(food_name = 'caesar salad')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.




caesar salad



,food,food_start,food_end,quantity,quantity_start,quantity_end,unit,unit_start,unit_end
0,Caesar salad,0,12,0,0,0,0,0,0


In [ ]:
ner_food(food_name = '3 apples')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.




3 apples



,food,food_start,food_end,quantity,quantity_start,quantity_end,unit,unit_start,unit_end
0,Apples,1,8,3,0,1,0,0,0


In [69]:
def split_text(food_name):

    pipe = pipeline("ner", model="davanstrien/deberta-v3-base_fine_tuned_food_ner")

    result = pipe(food_name)

    result = [entity for entity in result if entity['score'] >= 0.3]

    # Iterate over the results to merge consecutive rows with the same entity
    sentences=[]
    end_sentence = len(food_name)

    for i in range(len(result) - 1, 0, -1):
        current_entity = result[i]["entity"]
        previous_entity = result[i - 1]["entity"].split('-')[1]

        if result[i]["start"] != result[i-1]["end"]:
            start_index = result[i-1]["end"]
            end_index = result[i]["start"]

            if ' and' in food_name[start_index:end_index] or any([punc in food_name[start_index:end_index] for punc in string.punctuation]):
                sentences.append((food_name[end_index:end_sentence],end_index))
                end_sentence = start_index
                #split da dataframe para uma variável? depois corro a função na variável e junto os outputs das variáveis existentes?
                #pré-definição de 10 variáveis? Mesmo com NaN posso depois faço drop dos NaN...
    sentences.append((food_name[0:end_sentence],0))

    return sentences[::-1]


In [70]:
split_text('potato soup, also a caesar salad and also 3 grams of tree apples')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[('potato soup', 0), (' caesar salad', 19), (' 3 grams of tree apples', 41)]

In [89]:
pipe = pipeline("ner", model="davanstrien/deberta-v3-base_fine_tuned_food_ner")

result = pipe('caeser salad and 100 gr steak')

result = pipe('caeser salad')
result = pipe('100 gr steak')
result


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'U-QUANTITY',
  'score': 0.9888438,
  'index': 1,
  'word': '▁100',
  'start': 0,
  'end': 3},
 {'entity': 'U-UNIT',
  'score': 0.9795497,
  'index': 2,
  'word': '▁gr',
  'start': 3,
  'end': 6},
 {'entity': 'U-FOOD',
  'score': 0.9774789,
  'index': 3,
  'word': '▁steak',
  'start': 6,
  'end': 12}]

In [ ]:
split_text('potato soup, also a caesar salad and also 3 grams of tree apples')[0]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'potato soup'

In [58]:
def output(text):

    output = []

    for food, i in split_text(text):
        df = ner_food(food)
        df.iloc[:,1:]
        final = pd.concat(output)

    return final



In [61]:
df = output('potato soup, also a caesar salad and also 3 grams of tree apples')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.




potato soup



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.




 caesar salad



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.




 3 grams of tree apples



In [62]:
df

,food,food_start,food_end,quantity,quantity_start,quantity_end,unit,unit_start,unit_end
0,Potato soup,0,11,-1,-1,-1,-1,-1,-1
0,Caesar salad,1,13,-1,-1,-1,-1,-1,-1
0,Tree apples,11,23,3,1,2,grams,2,8


In [64]:
df.food_end.cumsum()

0    11
0    24
0    47
Name: food_end, dtype: int64

In [158]:
pipe = pipeline("ner", model="davanstrien/deberta-v3-base_fine_tuned_food_ner")


food_name = 'potato soup,  caesar salad and 3 apples'
result = pipe(food_name)

result = [entity for entity in result if entity['score'] >= 0.3]
print(result)

# Iterate over the results to merge consecutive rows with the same entity
for i in range(len(result) - 1, 0, -1):
    current_entity = result[i]["entity"]
    previous_entity = result[i - 1]["entity"].split('-')[1]

    if current_entity.split('-')[1] == previous_entity:
        if current_entity.split('-')[0] == "U":
            None
        if current_entity.split('-')[0] == "I" or current_entity.split('-')[0] == "L":
            # Append the word from the row below to the "word" column
            result[i - 1]["word"] += result[i]["word"]

            # Update the "end" value of the first row with the "end" value of the row below
            result[i - 1]["end"] = result[i]["end"]

            # Delete the current row
            del result[i]

a=pd.DataFrame(result)
a


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-FOOD', 'score': 0.9947789, 'index': 1, 'word': '▁potato', 'start': 0, 'end': 6}, {'entity': 'L-FOOD', 'score': 0.9952749, 'index': 2, 'word': '▁soup', 'start': 6, 'end': 11}, {'entity': 'B-FOOD', 'score': 0.9957991, 'index': 4, 'word': '▁caesar', 'start': 13, 'end': 20}, {'entity': 'L-FOOD', 'score': 0.9973519, 'index': 5, 'word': '▁salad', 'start': 20, 'end': 26}, {'entity': 'U-QUANTITY', 'score': 0.9414271, 'index': 7, 'word': '▁3', 'start': 30, 'end': 32}, {'entity': 'U-FOOD', 'score': 0.9915107, 'index': 8, 'word': '▁apples', 'start': 32, 'end': 39}]


,entity,score,index,word,start,end
0,B-FOOD,0.994779,1,▁potato▁soup,0,11
1,B-FOOD,0.995799,4,▁caesar▁salad,13,26
2,U-QUANTITY,0.941427,7,▁3,30,32
3,U-FOOD,0.991511,8,▁apples,32,39


In [159]:
sentences=[]
end_sentence = len(food_name)

for i in range(len(result) - 1, 0, -1):
    current_entity = result[i]["entity"]
    previous_entity = result[i - 1]["entity"].split('-')[1]

    if result[i]["start"] != result[i-1]["end"]:
        start_index = result[i-1]["end"]
        end_index = result[i]["start"]

        if ' and' in food_name[start_index:end_index] or any([punc in food_name[start_index:end_index] for punc in string.punctuation]):
            sentences.append(end_sentence)
            end_sentence = start_index
sentences.append(end_sentence)
sentences = sentences[::-1]
sentences

[11, 26, 39]

In [168]:
a=pd.DataFrame(result)

output = []
for end in sentences:
    i = a[a["end"]==end].index[0]
    output.append([])
    for row in range(i+1):
        output[-1].append(a.loc[row].to_dict())
    a = a.loc[i+1:].reset_index(drop=True)
len(output)

3

In [169]:
output

[[{'entity': 'B-FOOD',
   'score': 0.9947788715362549,
   'index': 1,
   'word': '▁potato▁soup',
   'start': 0,
   'end': 11}],
 [{'entity': 'B-FOOD',
   'score': 0.9957991242408752,
   'index': 4,
   'word': '▁caesar▁salad',
   'start': 13,
   'end': 26}],
 [{'entity': 'U-QUANTITY',
   'score': 0.9414271116256714,
   'index': 7,
   'word': '▁3',
   'start': 30,
   'end': 32},
  {'entity': 'U-FOOD',
   'score': 0.9915106892585754,
   'index': 8,
   'word': '▁apples',
   'start': 32,
   'end': 39}]]

In [141]:
output

[{'entity': {0: 'B-FOOD'},
  'score': {0: 0.9947788715362549},
  'index': {0: 1},
  'word': {0: '▁potato▁soup'},
  'start': {0: 0},
  'end': {0: 11}},
 {'entity': {1: 'B-FOOD'},
  'score': {1: 0.9957991242408752},
  'index': {1: 4},
  'word': {1: '▁caesar▁salad'},
  'start': {1: 13},
  'end': {1: 26}},
 {'entity': {2: 'U-QUANTITY', 3: 'U-FOOD'},
  'score': {2: 0.9414271116256714, 3: 0.9915106892585754},
  'index': {2: 7, 3: 8},
  'word': {2: '▁3', 3: '▁apples'},
  'start': {2: 30, 3: 32},
  'end': {2: 32, 3: 39}}]

In [144]:
a=pd.DataFrame(result)
a

,entity,score,index,word,start,end
0,B-FOOD,0.994779,1,▁potato▁soup,0,11
1,B-FOOD,0.995799,4,▁caesar▁salad,13,26
2,U-QUANTITY,0.941427,7,▁3,30,32
3,U-FOOD,0.991511,8,▁apples,32,39


In [149]:
a.loc[0].to_dict()

{'entity': {0: 'B-FOOD'},
 'score': {0: 0.9947788715362549},
 'index': {0: 1},
 'word': {0: '▁potato▁soup'},
 'start': {0: 0},
 'end': {0: 11}}

In [178]:
pd.concat([ner_food(o) for o in output])



potato soup,  caesar salad and 3 apples



potato soup,  caesar salad and 3 apples



potato soup,  caesar salad and 3 apples



,food,food_start,food_end,quantity,quantity_start,quantity_end,unit,unit_start,unit_end
0,Potato soup,0,11,-1,-1,-1,-1,-1,-1
0,Caesar salad,13,26,-1,-1,-1,-1,-1,-1
0,Apples,32,39,3,30,32,-1,-1,-1
